In [ ]:
%cd ..
%pip install -e .

In [1]:
import random

from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN0

from src.models.train_model import train
from src.models.train_model import train_conf_pen
from src.models.train_model import random_splits

from src.models.evaluate_model import evaluate0
from src.models.evaluate_model import evaluate1

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from sklearn.manifold import TSNE

from src.models.evaluate_model import test

In [9]:
def make_preg_loss(L_cls, L_preg, mu, A_hat):
    """  """
    def l(data, pred):
        L_cls = L_cls(pred[data.train_mask], data.y[data.train_mask])
        L_preg = L_preg(pred[data.train_mask], A_hat@pred[data.reg_mask])
        return  + mu * L_preg(x,y)
    return l


def make_confidence_penalty_loss(L_cls, beta):
    """  """
    def l(data, pred):
        L_cls = L_cls(pred[data.train_mask], data.y[data.train_mask])
        H = -torch.sum(pred[data.reg_mask] * torch.log2(pred[data.reg_mask])) / pred.shape[0]

        return L_cls - beta * H
    return l

In [10]:
L_cls = lambda x, y: F.nll_loss(torch.log(x), y),
L_preg = lambda x, y: F.cross_entropy(x, y),


In [18]:
l = make_preg_loss(L_cls, L_preg, 0.2)

TypeError: make_preg_loss() missing 1 required positional argument: 'A_hat'

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
# no normalization, using n != 0 and m!= 0 instead
# doesnt work, just use normalization instead
# dataset = Planetoid(root='data/Planetoid', name='Cora')

data = dataset[0].to(device)
data = random_splits(data, 50, 20)
splits = data.train_mask, data.val_mask, data.test_mask

Processing...
Done!


In [17]:
def set_reg_mask(data, A):
    reg_indeces = random.sample(range(data.x.shape[0]), k=A)
    reg_mask = torch.zeros(len(data.y), dtype=torch.bool)
    for i in reg_indeces:
        reg_mask[i] = True
    data.reg_mask = reg_mask
    return data

data = set_reg_mask(data, 20)
print(data.reg_mask.sum())

tensor(20)


In [28]:
from src.models.reg import compute_a_hat

def make_preg_loss(L_cls, L_preg, mu, A_hat):
    """  """
    def l(data, pred):
        l_cls = L_cls(pred[data.train_mask], data.y[data.train_mask])
        l_preg = L_preg(pred[data.reg_mask], (A_hat@pred)[data.reg_mask])
        return l_cls + mu * l_preg
    return l


def make_confidence_penalty_loss(L_cls, beta):
    """  """
    def l(data, pred):
        l_cls = L_cls(pred[data.train_mask], data.y[data.train_mask])
        H = -torch.sum(pred[data.reg_mask] * torch.log2(pred[data.reg_mask])) / pred.shape[0]

        return l_cls - beta * H
    return l


a_hat = compute_a_hat(data)

L_cls = lambda x, y: F.nll_loss(torch.log(x), y)
L_preg = lambda x, y: F.cross_entropy(x, y)

l = make_preg_loss(L_cls, L_preg, 0.2, a_hat)


gcn_model = GCN0(num_node_features=dataset.num_node_features,
                num_classes=dataset.num_classes) \
                .to(device)

pred = gcn_model(data)
comped_l = l(data, pred)
print(comped_l)



tensor(nan, grad_fn=<AddBackward0>)
